In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import kurtosis, skew, norm, expon

import aiohttp, pandas as pd, io
from shadowing.utils import windows

# Reminders on kurtosis and skewness
## Kurtosis
Kurtosis is a measure of the "tailedness" of the probability distribution of a real-valued random variable. It is a descriptor of the shape of a probability distribution and provides information about the tails of the distribution.
It requires the calculation of the fourth central moment.
$$\beta_2 = \frac{\mu_4}{\sigma^4}=\mathbb E\left[\left(\frac{X-\mu}{\sigma}\right)^4\right]$$
where $\mu_4$ is the fourth central moment and $\sigma$ is the standard deviation.
## Skewness
Skewness is a measure of the asymmetry of the probability distribution of a real-valued random variable about its mean. The skewness value can be positive, zero, negative, or undefined.
It requires the calculation of the third central moment.
$$\gamma_1 = \frac{\mu_3}{\sigma^3}=\mathbb E\left[\left(\frac{X-\mu}{\sigma}\right)^3\right]$$
where $\mu_3$ is the third central moment and $\sigma$ is the standard deviation.
## Relationship between kurtosis and skewness
The kurtosis is related to the skewness by the following formula:
$$\beta_2 = \gamma_1^2 + 1$$
## Kurtosis and skewness of some distributions
- Normal distribution: $\beta_2=3$ and $\gamma_1=0$
- Uniform distribution: $\beta_2=-\frac{6}{5}$ and $\gamma_1=0$
- Exponential distribution: $\beta_2=9$ and $\gamma_1=2$
- Cauchy distribution: $\beta_2=\infty$ and $\gamma_1=\infty$

## Visualizing kurtosis and skewness

In [ ]:
normal_sample = np.random.normal(0, 1, 10_000)
plt.figure(figsize=(10, 5))
plt.hist(normal_sample, density=True, alpha=0.5, label="Normal Distribution", bins=100)
plt.title("Histogram of normal distribution")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.plot(
    np.linspace(np.min(normal_sample), np.max(normal_sample), 100),
    norm.pdf(np.linspace(np.min(normal_sample), np.max(normal_sample), 100)),
    label="PDF",
    color="black",
)
plt.legend(loc="upper right")
plt.axis(xmin=np.min(normal_sample), xmax=np.max(normal_sample))
plt.show()
print("Kurtosis of normal distribution: ", kurtosis(normal_sample, fisher=False))
print("Skewness of normal distribution: ", skew(normal_sample))

In [ ]:
expo_sample = np.random.exponential(1, 10_000)
plt.figure(figsize=(10, 5))
plt.hist(expo_sample, density=True, alpha=0.5, label="Exponential Distribution", bins=100)
plt.title("Histogram of exponential distribution")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.plot(
    np.linspace(np.min(expo_sample), np.max(expo_sample), 100),
    expon.pdf(np.linspace(np.min(expo_sample), np.max(expo_sample), 100)),
    label="PDF",
    color="black",
)
plt.legend(loc="upper right")
plt.axis(xmin=np.min(expo_sample), xmax=np.max(expo_sample))
plt.show()
print("Kurtosis of exponential distribution: ", kurtosis(expo_sample, fisher=False))
print("Skewness of exponential distribution: ", skew(expo_sample))

# Black-Scholes model
We are now going to compare the Black-Scholes model with the Scattering Spectra model. The Black-Scholes model is a mathematical model for the dynamics of a financial market containing derivative investment instruments. From the Black-Scholes model, we can derive the following formula for the price of a call option:
$$C(S,t)=S_0N(d_1)-Ke^{-r(T-t)}N(d_2)$$
where:
- $C(S,t)$ is the price of the call option;
- $S_0$ is the current price of the underlying asset;
- $K$ is the strike price;
- $r$ is the risk-free interest rate;
- $N$ is the cumulative distribution function of the standard normal distribution;
- $\displaystyle d_1=\frac{\ln\left(\frac{S_0}{K}\right)+(r+\frac{\sigma^2}{2})(T-t)}{\sigma\sqrt{T-t}}$;
- $\displaystyle d_2=d_1-\sigma\sqrt{T-t}$;
- $\sigma$ is the volatility of the underlying asset;
- $T$ is the time to maturity.


In [ ]:
# Settings constants
T = 1
N = 1000
K = 1000
r = 0.05
S = np.linspace(100, 2 * K, N)
sigma = 0.2
d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(S, S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2), label="Call Option")
plt.legend(loc="upper right")
plt.title("Black-Scholes Call Option")
plt.xlabel("Stock Price")
plt.ylabel("Option Price")
plt.show()

To use this model, we need to also simulate $S$ using either past data and fitting a model or using a stochastic process.

First, we are goign to use a Brownian motion:
$$S_t=S_0\mathrm e^{(\mu-\frac{\sigma^2}{2})t+\sigma W_t}$$

In [ ]:
brownian_motion = np.cumsum(np.random.normal(0, 1, N))
S0 = 100

price = S0 * np.exp(
    (r - 0.5 * sigma**2) * np.linspace(0, T, N) + sigma * brownian_motion
)

plt.figure(figsize=(10, 5))
plt.plot(np.linspace(0, T, N), price, label="Brownian Motion")
plt.legend(loc="upper right")
plt.title("Brownian Motion")
plt.xlabel("Time")
plt.ylabel("Value")
plt.show()

# Scaterring spectra model
This time, we are going to use the scattering spectra model.
This model can be used with time series that have stationary increments.

For this test, I will use real data from the S&P500 index.

In [ ]:
async with aiohttp.ClientSession() as session:
    async with session.get("https://financial-data.shriimpe.fr/api/ticker/history/^GSPC?diff=True&operator=log&fields=Close", headers={"Accept": "python/pickle"}) as response:
        df: pd.DataFrame = pd.read_pickle(io.BytesIO(await response.content.read()))

df.head()

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("S&P 500 log prices")
plt.plot(df["Close"])
plt.show()

In [ ]:
context = windows(df["Close"].values, 100, 10)
print("Context shape:", context.shape)

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Scattering spectra from S&P 500 log prices")
plt.plot(range(len(context[1, :])), context[1, :])
plt.show()